In [163]:
import datetime
import re

import pandas as pd
import numpy as np

import statsmodels.api as sm
import matplotlib.pyplot as plt

In [164]:
df = pd.read_csv('transformer_data2.csv')
df.head()

,date,quantity,suppliers_currency,transformer_unit_price,dry_or_oil,rated_power_kVA,primary_winding,secondary_winding,no_load_loss,full_load_loss_75,full_load_loss_120,rated_volt_primary_side
0,28-April-21,6.0,EUR,708000.0,Oil,19080.0,Copper,NaN,NaN,NaN,NaN,33.0
1,22.04.2021,1.0,NaN,NaN,Oil,19080.0,Copper,NaN,17000.0,90000.0,NaN,33750.0
2,22.04.2021,NaN,NaN,NaN,Oil,19080.0,Copper,Copper,17000.0,90000.0,NaN,33750.0
3,14/03/2023,4.0,€,235000.0,Oil,11000.0,Copper,Copper,NaN,NaN,NaN,NaN
4,14/03/2023,NaN,NaN,NaN,Oil,11000.0,Copper,NaN,7800.0,88000.0,NaN,22.0


In [165]:
original_dates = list(df['date'].unique())
gpt_parsed_dates = [
    '2021-04-28',
    '2021-04-22',
    '2023-03-14',
    '2023-03-14',
    '2021-09-30',
    '2020-12-02',
    '2020-12-09',
    '2020-06-03',
    '2021-08-12',
    '2022-03-17',
    '2021-06-08',
    '2021-12-14',
    '2022-09-09',
    '2023-02-15',
    '2022-09-13',
    '2022-05-13',
    '2022-02-16',
    '2022-03-11',
    '2022-08-05',
    '2022-08-18',
    '2022-07-19',
    '2020-10-06',
    '2021-02-11',
    '2023-03-27',
    '2023-03-07',
    '2021-07-19',
    '2021-07-26',
    '2022-08-02',
    '2021-09-08',
    '2021-06-07',
    None,
    '2022-08-04',
    '2023-03-17',
    '2021-08-13',
    '2022-11-09',
    '2020-12-08',
    '2020-10-05',
    '2022-02-07',
    '2020-08-31',
    '2020-06-10',
    '2023-02-08',
    '2023-02-10',
    '2021-09-09',
    '2021-11-03',
    '2021-07-29',
    '2022-08-22',
    '2021-09-02',
    '2021-08-26',
    '2022-08-15',
    '2022-03-09',
    '2022-02-01',
    '2022-04-06',
    '2021-12-15',
    '2022-05-18',
    '2022-09-12',
    '2021-10-06',
    '2022-08-10',
    '2021-06-17',
    '2022-03-18',
    '2021-02-04',
    '2021-06-10',
    '2022-09-09',
    '2022-12-06',
    '2020-12-01',
    '2020-07-05',
    '2020-11-28',
    '2021-06-01',
    '2021-12-14',
    '2021-12-14',
    '2021-09-27',
    '2021-05-04',
    '2021-09-10',
    '2021-08-31',
    '2022-05-24',
    '2020-09-11'
]
matching_dates = dict(zip(original_dates, gpt_parsed_dates))
df['date'] = df['date'].apply(lambda x: matching_dates[x])

In [166]:
df['suppliers_currency'].unique()
df['suppliers_currency'] = df['suppliers_currency'].apply(lambda x: 'EUR' if not pd.isna(x) else None)

In [167]:
df['transformer_unit_price'] = df['transformer_unit_price'].apply(lambda x: x if not pd.isna(x) and (x>1000 and x<1000000) else None)

In [168]:
df['dry_or_oil'].unique()
df['dry_or_oil'] = df['dry_or_oil'].apply(lambda x: {'Oil': 'Oil', 'Dry': 'Dry', 'prepreg': 'Dry', 'vergossen': 'Oil', 'F': None, 'dry': 'Dry'}[x] if not pd.isna(x) else None)

In [169]:
df['rated_power_kVA'] = df['rated_power_kVA'].apply(lambda x: x if not pd.isna(x) and x>100 else None)

In [170]:
df['primary_winding'] = df['primary_winding'].apply(lambda x: {'Copper': 'Copper', 'Aluminium': 'Aluminum', 'aluminum': "Aluminum", 'Aluminum': 'Aluminum'}.get(x, None))

In [171]:
df['secondary_winding'] = df['secondary_winding'].apply(lambda x: {'Copper': 'Copper', 'Aluminium': 'Aluminum', 'aluminum': 'Aluminum', 'Aluminum': 'Aluminum'}.get(x, None))

In [172]:
df['no_load_loss'] = df['no_load_loss'].apply(lambda x: x if not pd.isna(x) and x>100 else x*1000 if x==100 else None)

In [173]:
df['full_load_loss_75'] = df['full_load_loss_75'].apply(lambda x: x if not pd.isna(x) and x>21 else x*1000 if x==21 else None)

In [174]:
df['full_load_loss_120'] = df['full_load_loss_120'].apply(lambda x: x if not pd.isna(x) and x>120 else x*1000 if x==120 else None)

In [175]:
df['rated_volt_primary_side'] = df['rated_volt_primary_side'].apply(lambda x: x if not pd.isna(x) and x>100 else x*1000 if (x<100 and x>1) else None)

In [176]:
df.to_csv('./output/v1.csv')